In [1]:
import pandas as pd

import numpy as np

from sklearn import datasets


from evidently.report import Report

from evidently.metrics import DataDriftTable

from evidently.metrics import DatasetDriftMetric

/home/toducthanh155/.local/share/virtualenvs/mlops-mara-sample-public-DwtKEUlZ/lib/python3.9/site-packages/umap/distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/home/toducthanh155/.local/share/virtualenvs/mlops-mara-sample-public-DwtKEUlZ/lib/python3.9/site-packages/umap/distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.

In [2]:
adult_data = datasets.fetch_openml(name='adult', version=2, as_frame='auto')

/home/toducthanh155/.local/share/virtualenvs/mlops-mara-sample-public-DwtKEUlZ/lib/python3.9/site-packages/sklearn/datasets/_openml.py:968: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


In [14]:
type(adult_data)

sklearn.utils._bunch.Bunch

In [4]:
adult_data.keys()

dict_keys(['data', 'target', 'frame', 'categories', 'feature_names', 'target_names', 'DESCR', 'details', 'url'])

In [10]:
adult_data.data.equals(adult_data.frame)

False

In [12]:
adult = adult_data.frame
adult.head(3)

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,class
0,25.0,Private,226802.0,11th,7.0,Never-married,Machine-op-inspct,Own-child,Black,Male,0.0,0.0,40.0,United-States,<=50K
1,38.0,Private,89814.0,HS-grad,9.0,Married-civ-spouse,Farming-fishing,Husband,White,Male,0.0,0.0,50.0,United-States,<=50K
2,28.0,Local-gov,336951.0,Assoc-acdm,12.0,Married-civ-spouse,Protective-serv,Husband,White,Male,0.0,0.0,40.0,United-States,>50K


In [13]:
adult.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48842 entries, 0 to 48841
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype   
---  ------          --------------  -----   
 0   age             48842 non-null  float64 
 1   workclass       46043 non-null  category
 2   fnlwgt          48842 non-null  float64 
 3   education       48842 non-null  category
 4   education-num   48842 non-null  float64 
 5   marital-status  48842 non-null  category
 6   occupation      46033 non-null  category
 7   relationship    48842 non-null  category
 8   race            48842 non-null  category
 9   sex             48842 non-null  category
 10  capital-gain    48842 non-null  float64 
 11  capital-loss    48842 non-null  float64 
 12  hours-per-week  48842 non-null  float64 
 13  native-country  47985 non-null  category
 14  class           48842 non-null  category
dtypes: category(9), float64(6)
memory usage: 2.7 MB


In [17]:
adult.education.unique()

['11th', 'HS-grad', 'Assoc-acdm', 'Some-college', '10th', ..., 'Assoc-voc', '9th', '12th', '1st-4th', 'Preschool']
Length: 16
Categories (16, object): ['10th', '11th', '12th', '1st-4th', ..., 'Masters', 'Preschool', 'Prof-school', 'Some-college']

In [18]:
len(adult)

48842

In [19]:
adult_ref = adult[~adult.education.isin(['Some-college', 'HS-grad', 'Bachelors'])]
adult_cur = adult[adult.education.isin(['Some-college', 'HS-grad', 'Bachelors'])]

In [20]:
len(adult_ref), len(adult_cur)

(14155, 34687)

In [21]:
adult_ref.info()

<class 'pandas.core.frame.DataFrame'>
Index: 14155 entries, 0 to 48837
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype   
---  ------          --------------  -----   
 0   age             14155 non-null  float64 
 1   workclass       13259 non-null  category
 2   fnlwgt          14155 non-null  float64 
 3   education       14155 non-null  category
 4   education-num   14155 non-null  float64 
 5   marital-status  14155 non-null  category
 6   occupation      13253 non-null  category
 7   relationship    14155 non-null  category
 8   race            14155 non-null  category
 9   sex             14155 non-null  category
 10  capital-gain    14155 non-null  float64 
 11  capital-loss    14155 non-null  float64 
 12  hours-per-week  14155 non-null  float64 
 13  native-country  13861 non-null  category
 14  class           14155 non-null  category
dtypes: category(9), float64(6)
memory usage: 902.6 KB


In [23]:
adult_cur.head(2)

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,class
1,38.0,Private,89814.0,HS-grad,9.0,Married-civ-spouse,Farming-fishing,Husband,White,Male,0.0,0.0,50.0,United-States,<=50K
3,44.0,Private,160323.0,Some-college,10.0,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688.0,0.0,40.0,United-States,>50K


In [24]:
adult_cur.iloc[:2000, 3:5] = np.nan

In [25]:
adult_cur.info()

<class 'pandas.core.frame.DataFrame'>
Index: 34687 entries, 1 to 48841
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype   
---  ------          --------------  -----   
 0   age             34687 non-null  float64 
 1   workclass       32784 non-null  category
 2   fnlwgt          34687 non-null  float64 
 3   education       32687 non-null  category
 4   education-num   32687 non-null  float64 
 5   marital-status  34687 non-null  category
 6   occupation      32780 non-null  category
 7   relationship    34687 non-null  category
 8   race            34687 non-null  category
 9   sex             34687 non-null  category
 10  capital-gain    34687 non-null  float64 
 11  capital-loss    34687 non-null  float64 
 12  hours-per-week  34687 non-null  float64 
 13  native-country  34124 non-null  category
 14  class           34687 non-null  category
dtypes: category(9), float64(6)
memory usage: 2.2 MB


In [26]:
data_drift_dataset_report = Report(metrics=[

    DatasetDriftMetric(),

    DataDriftTable(),    

])

data_drift_dataset_report.run(reference_data=adult_ref, current_data=adult_cur)

data_drift_dataset_report

In [28]:
import pandas as pd
import numpy as np

from sklearn.datasets import fetch_california_housing

from evidently import ColumnMapping

from evidently.report import Report
from evidently.metrics.base_metric import generate_column_metrics
from evidently.metric_preset import DataDriftPreset, TargetDriftPreset
from evidently.metrics import *

from evidently.test_suite import TestSuite
from evidently.tests.base_test import generate_column_tests
from evidently.test_preset import DataStabilityTestPreset, NoTargetPerformanceTestPreset
from evidently.tests import *

In [29]:
data = fetch_california_housing(as_frame=True)
housing_data = data.frame

In [31]:
housing_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20640 entries, 0 to 20639
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   MedInc       20640 non-null  float64
 1   HouseAge     20640 non-null  float64
 2   AveRooms     20640 non-null  float64
 3   AveBedrms    20640 non-null  float64
 4   Population   20640 non-null  float64
 5   AveOccup     20640 non-null  float64
 6   Latitude     20640 non-null  float64
 7   Longitude    20640 non-null  float64
 8   MedHouseVal  20640 non-null  float64
dtypes: float64(9)
memory usage: 1.4 MB


In [32]:
housing_data.rename(columns={'MedHouseVal': 'target'}, inplace=True)
housing_data['prediction'] = housing_data['target'].values + np.random.normal(0, 5, housing_data.shape[0])

In [33]:
reference = housing_data.sample(n=5000, replace=False)
current = housing_data.sample(n=5000, replace=False)

In [34]:
report = Report(metrics=[
    DataDriftPreset(), 
])

report.run(reference_data=reference, current_data=current)
report

In [35]:
import os
import pandas as pd
import glob
path = "data/captured_data/phase-1/prob-1"
files = glob.glob(path + "/*.parquet")
len(files)

59

In [36]:
path = "data/captured_data/phase-1/prob-2"
files = glob.glob(path + "/*.parquet")
len(files)

31

In [43]:
path = "data/captured_data/phase-1/prob-1"
files = glob.glob(path + "/*.parquet")
for i, file in enumerate(files):
    print(f"Processing {os.path.basename(file)}")
    os.rename(file, os.path.join(path, f"prob1_{i}.parquet"))

Processing prob_47.parquet
Processing prob_0.parquet
Processing prob_2.parquet
Processing prob_51.parquet
Processing prob_52.parquet
Processing prob_14.parquet
Processing prob_23.parquet
Processing prob_37.parquet
Processing prob_46.parquet
Processing prob_19.parquet
Processing prob_34.parquet
Processing prob_1.parquet
Processing prob_28.parquet
Processing prob_38.parquet
Processing prob_24.parquet
Processing prob_50.parquet
Processing prob_36.parquet
Processing prob_10.parquet
Processing prob_3.parquet
Processing prob_32.parquet
Processing prob_8.parquet
Processing prob_21.parquet
Processing prob_40.parquet
Processing prob_13.parquet
Processing prob_33.parquet
Processing prob_26.parquet
Processing prob_58.parquet
Processing prob_6.parquet
Processing prob_16.parquet
Processing prob_42.parquet
Processing prob_25.parquet
Processing prob_30.parquet
Processing prob_43.parquet
Processing prob_5.parquet
Processing prob_54.parquet
Processing prob_56.parquet
Processing prob_18.parquet
Processi

In [57]:
df = pd.read_parquet(os.path.join(path, "prob1_0.parquet"))

In [50]:
len(df)

1000

In [58]:
df.head()

,feature1,feature2,feature3,feature4,feature5,feature6,feature7,feature8,feature9,feature10,feature11,feature12,feature13,feature14,feature15,feature16,is_drift,batch_id
0,-1,12,28.71,37341,34.764269,-82.233754,12085,34.310798,-85.949851,23.232385,23,2,6,0.925699,1.657646,64.008021,0,18
1,7,11,4.56,48314,42.234590,-83.858655,129683,41.835677,-82.599450,21.750394,15,0,6,1.151222,1.674276,86.896538,0,18
2,197,4,43.01,62221,32.594352,-89.977493,95666,38.438088,-89.192691,15.009756,11,1,5,1.019024,1.355796,81.926642,0,18
3,288,9,8.73,31907,28.381704,-84.570469,173903,33.399397,-85.824090,23.088269,17,3,5,0.871280,1.601172,75.423162,0,18
4,212,2,52.54,61071,39.724923,-89.094662,14381,42.227150,-90.645944,23.875300,4,0,4,1.051622,1.199672,79.158009,0,18


In [52]:
df["is_drift"].unique()

array([0])

In [59]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 18 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   feature1   1000 non-null   int16  
 1   feature2   1000 non-null   int8   
 2   feature3   1000 non-null   float64
 3   feature4   1000 non-null   int64  
 4   feature5   1000 non-null   float64
 5   feature6   1000 non-null   float64
 6   feature7   1000 non-null   int64  
 7   feature8   1000 non-null   float64
 8   feature9   1000 non-null   float64
 9   feature10  1000 non-null   float64
 10  feature11  1000 non-null   int64  
 11  feature12  1000 non-null   int64  
 12  feature13  1000 non-null   int64  
 13  feature14  1000 non-null   float64
 14  feature15  1000 non-null   float64
 15  feature16  1000 non-null   float64
 16  is_drift   1000 non-null   int64  
 17  batch_id   1000 non-null   int64  
dtypes: float64(9), int16(1), int64(7), int8(1)
memory usage: 128.1 KB


In [74]:
import pickle
with open("data/train_data/phase-1/prob-1/category_index.pickle", "rb") as f:
    category_index = pickle.load(f)

In [80]:
pd.CategoricalDtype(category_index["feature1"], ordered=True)

CategoricalDtype(categories=['Academic librarian', 'Accommodation manager',
                  'Accountant, chartered', 'Accountant, chartered certified',
                  'Accountant, chartered management', 'Accounting technician',
                  'Actuary', 'Acupuncturist', 'Administrator',
                  'Administrator, Civil Service',
                  ...
                  'Tree surgeon', 'Video editor', 'Volunteer coordinator',
                  'Warden/ranger', 'Warehouse manager',
                  'Waste management officer', 'Water engineer',
                  'Water quality scientist', 'Web designer',
                  'Wellsite geologist'],
, ordered=True)

In [83]:
category_index["feature1"].to_list()[337]

'Probation officer'

In [65]:
raw_data = pd.read_parquet("data/raw_data/phase-1/prob-1/raw_train.parquet")
raw_data[raw_data["feature1"] == "Academic librarian"].head()

,feature1,feature2,feature3,feature4,feature5,feature6,feature7,feature8,feature9,feature10,feature11,feature12,feature13,label,feature14,feature15,feature16
51609,Academic librarian,misc_net,8.91,93263,39.700825,-120.653458,19582,35.095892,-120.058334,40.712786,1,6,1,1,6.877563,1.011988,62.402688
51610,Academic librarian,misc_net,843.93,93263,34.282101,-120.066530,19582,36.137244,-119.143338,41.025356,0,6,1,1,5.955077,1.040456,49.747494
51611,Academic librarian,misc_net,830.62,93263,33.847559,-121.651197,19582,35.044830,-119.326952,41.393418,0,6,1,1,8.113371,1.013194,58.303757
51612,Academic librarian,gas_transport,729.89,93263,38.625045,-116.573543,19582,35.206591,-120.003625,40.805682,8,6,1,1,8.409083,0.869041,62.568539
51613,Academic librarian,shopping_net,898.96,93263,33.150174,-120.329113,19582,35.204861,-118.384329,40.604671,22,0,1,1,6.493531,1.666962,58.415303


In [66]:
train_data = pd.read_parquet("data/train_data/phase-1/prob-1/train_x.parquet")
train_data.head()

,feature1,feature2,feature3,feature4,feature5,feature6,feature7,feature8,feature9,feature10,feature11,feature12,feature13,feature14,feature15,feature16
0,337,0,138.71,44121,39.354345,-83.372015,769341,41.359975,-80.587306,78.529290,10,1,1,6.406360,0.284834,67.097437
1,365,12,6.51,66442,35.920414,-98.480588,14283,39.439880,-96.938809,45.234461,18,2,2,6.800447,0.779245,66.892996
2,184,6,3.34,42071,36.155589,-88.412050,29655,37.220467,-88.393033,43.063620,22,2,2,8.342428,0.603699,66.288586
3,390,4,180.17,30067,31.949179,-82.265798,302309,33.430411,-84.920539,26.093487,0,3,1,8.697487,0.472254,63.834991
4,365,10,8.92,66442,40.881803,-99.983715,14283,38.770383,-95.877356,44.831443,16,5,2,6.544843,0.429824,62.746348


In [67]:
category_index = {}
df = raw_data.copy()
for col in ["feature1"]:
    df[col] = df[col].astype("category")
    category_index[col] = df[col].cat.categories
    df["convert_numeric"] = df[col].cat.codes

In [68]:
category_index

{'feature1': Index(['Academic librarian', 'Accommodation manager', 'Accountant, chartered',
        'Accountant, chartered certified', 'Accountant, chartered management',
        'Accounting technician', 'Actuary', 'Acupuncturist', 'Administrator',
        'Administrator, Civil Service',
        ...
        'Tree surgeon', 'Video editor', 'Volunteer coordinator',
        'Warden/ranger', 'Warehouse manager', 'Waste management officer',
        'Water engineer', 'Water quality scientist', 'Web designer',
        'Wellsite geologist'],
       dtype='object', length=481)}

In [72]:
df[df["feature1"] == "Academic librarian"].head()

,feature1,feature2,feature3,feature4,feature5,feature6,feature7,feature8,feature9,feature10,feature11,feature12,feature13,label,feature14,feature15,feature16,convert_numeric
51609,Academic librarian,misc_net,8.91,93263,39.700825,-120.653458,19582,35.095892,-120.058334,40.712786,1,6,1,1,6.877563,1.011988,62.402688,0
51610,Academic librarian,misc_net,843.93,93263,34.282101,-120.066530,19582,36.137244,-119.143338,41.025356,0,6,1,1,5.955077,1.040456,49.747494,0
51611,Academic librarian,misc_net,830.62,93263,33.847559,-121.651197,19582,35.044830,-119.326952,41.393418,0,6,1,1,8.113371,1.013194,58.303757,0
51612,Academic librarian,gas_transport,729.89,93263,38.625045,-116.573543,19582,35.206591,-120.003625,40.805682,8,6,1,1,8.409083,0.869041,62.568539,0
51613,Academic librarian,shopping_net,898.96,93263,33.150174,-120.329113,19582,35.204861,-118.384329,40.604671,22,0,1,1,6.493531,1.666962,58.415303,0


In [88]:
with open("data/train_data/phase-1/prob-1/category_index.pickle", "rb") as f:
    category_index = pickle.load(f)

In [89]:
category_index

{'feature2': Index(['entertainment', 'food_dining', 'gas_transport', 'grocery_net',
        'grocery_pos', 'health_fitness', 'home', 'kids_pets', 'misc_net',
        'misc_pos', 'personal_care', 'shopping_net', 'shopping_pos', 'travel'],
       dtype='object'),
 'feature1': Index(['Academic librarian', 'Accommodation manager', 'Accountant, chartered',
        'Accountant, chartered certified', 'Accountant, chartered management',
        'Accounting technician', 'Actuary', 'Acupuncturist', 'Administrator',
        'Administrator, Civil Service',
        ...
        'Tree surgeon', 'Video editor', 'Volunteer coordinator',
        'Warden/ranger', 'Warehouse manager', 'Waste management officer',
        'Water engineer', 'Water quality scientist', 'Web designer',
        'Wellsite geologist'],
       dtype='object', length=481)}

In [106]:
dtype_cat = pd.CategoricalDtype(category_index["feature2"], ordered=True)

In [107]:
dtype_cat

CategoricalDtype(categories=['entertainment', 'food_dining', 'gas_transport',
                  'grocery_net', 'grocery_pos', 'health_fitness', 'home',
                  'kids_pets', 'misc_net', 'misc_pos', 'personal_care',
                  'shopping_net', 'shopping_pos', 'travel'],
, ordered=True)

In [95]:
curr_data[["feature2", "feature1"]]

,feature2,feature1
0,0,337
1,12,365
2,6,184
3,4,390
4,10,365
...,...,...
107355,11,288
107356,0,122
107357,7,340
107358,11,53


In [109]:
curr_data["cat2"] = pd.Categorical.from_codes(curr_data.feature2, dtype=dtype_cat)

In [111]:
curr_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 107360 entries, 0 to 107359
Data columns (total 17 columns):
 #   Column     Non-Null Count   Dtype   
---  ------     --------------   -----   
 0   feature1   107360 non-null  int16   
 1   feature2   107360 non-null  int8    
 2   feature3   107360 non-null  float64 
 3   feature4   107360 non-null  int64   
 4   feature5   107360 non-null  float64 
 5   feature6   107360 non-null  float64 
 6   feature7   107360 non-null  int64   
 7   feature8   107360 non-null  float64 
 8   feature9   107360 non-null  float64 
 9   feature10  107360 non-null  float64 
 10  feature11  107360 non-null  int64   
 11  feature12  107360 non-null  int64   
 12  feature13  107360 non-null  int64   
 13  feature14  107360 non-null  float64 
 14  feature15  107360 non-null  float64 
 15  feature16  107360 non-null  float64 
 16  cat2       107360 non-null  category
dtypes: category(1), float64(9), int16(1), int64(5), int8(1)
memory usage: 11.9 MB


In [113]:
curr_data.head(1)

,feature1,feature2,feature3,feature4,feature5,feature6,feature7,feature8,feature9,feature10,feature11,feature12,feature13,feature14,feature15,feature16,cat2
0,337,0,138.71,44121,39.354345,-83.372015,769341,41.359975,-80.587306,78.52929,10,1,1,6.40636,0.284834,67.097437,entertainment


In [117]:
ref_data = pd.read_parquet("data/train_data/phase-1/prob-1/train_x.parquet")
curr_data = pd.read_parquet(os.path.join(path, "prob1_0.parquet"))

In [118]:
curr_data.head(1)

,feature1,feature2,feature3,feature4,feature5,feature6,feature7,feature8,feature9,feature10,feature11,feature12,feature13,feature14,feature15,feature16,is_drift,batch_id
0,-1,12,28.71,37341,34.764269,-82.233754,12085,34.310798,-85.949851,23.232385,23,2,6,0.925699,1.657646,64.008021,0,18


In [119]:
ref_data.head(1)

,feature1,feature2,feature3,feature4,feature5,feature6,feature7,feature8,feature9,feature10,feature11,feature12,feature13,feature14,feature15,feature16
0,337,0,138.71,44121,39.354345,-83.372015,769341,41.359975,-80.587306,78.52929,10,1,1,6.40636,0.284834,67.097437


In [ ]:
# curr_data: new data
# ref_data: base data

In [120]:
ref_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 107360 entries, 0 to 107359
Data columns (total 16 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   feature1   107360 non-null  int16  
 1   feature2   107360 non-null  int8   
 2   feature3   107360 non-null  float64
 3   feature4   107360 non-null  int64  
 4   feature5   107360 non-null  float64
 5   feature6   107360 non-null  float64
 6   feature7   107360 non-null  int64  
 7   feature8   107360 non-null  float64
 8   feature9   107360 non-null  float64
 9   feature10  107360 non-null  float64
 10  feature11  107360 non-null  int64  
 11  feature12  107360 non-null  int64  
 12  feature13  107360 non-null  int64  
 13  feature14  107360 non-null  float64
 14  feature15  107360 non-null  float64
 15  feature16  107360 non-null  float64
dtypes: float64(9), int16(1), int64(5), int8(1)
memory usage: 11.8 MB


In [121]:
curr_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 18 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   feature1   1000 non-null   int16  
 1   feature2   1000 non-null   int8   
 2   feature3   1000 non-null   float64
 3   feature4   1000 non-null   int64  
 4   feature5   1000 non-null   float64
 5   feature6   1000 non-null   float64
 6   feature7   1000 non-null   int64  
 7   feature8   1000 non-null   float64
 8   feature9   1000 non-null   float64
 9   feature10  1000 non-null   float64
 10  feature11  1000 non-null   int64  
 11  feature12  1000 non-null   int64  
 12  feature13  1000 non-null   int64  
 13  feature14  1000 non-null   float64
 14  feature15  1000 non-null   float64
 15  feature16  1000 non-null   float64
 16  is_drift   1000 non-null   int64  
 17  batch_id   1000 non-null   int64  
dtypes: float64(9), int16(1), int64(7), int8(1)
memory usage: 128.1 KB


In [124]:
from evidently import ColumnMapping

numeric_columns = [
    "feature3",
    "feature4",
    "feature5",
    "feature6",
    "feature7",
    "feature8",
    "feature9",
    "feature10",
    "feature11",
    "feature12",
    "feature13",
    "feature14",
    "feature15",
    "feature16"
]

category_columns = [
    "feature2",
    "feature1"
]

column_mapping = ColumnMapping()

# column_mapping.target = target
# column_mapping.prediction = prediction
column_mapping.numerical_features = numeric_columns
column_mapping.categorical_features = category_columns

In [126]:
curr_data = curr_data.drop(["is_drift", "batch_id"], axis=1)
curr_data.head()

,feature1,feature2,feature3,feature4,feature5,feature6,feature7,feature8,feature9,feature10,feature11,feature12,feature13,feature14,feature15,feature16
0,-1,12,28.71,37341,34.764269,-82.233754,12085,34.310798,-85.949851,23.232385,23,2,6,0.925699,1.657646,64.008021
1,7,11,4.56,48314,42.234590,-83.858655,129683,41.835677,-82.599450,21.750394,15,0,6,1.151222,1.674276,86.896538
2,197,4,43.01,62221,32.594352,-89.977493,95666,38.438088,-89.192691,15.009756,11,1,5,1.019024,1.355796,81.926642
3,288,9,8.73,31907,28.381704,-84.570469,173903,33.399397,-85.824090,23.088269,17,3,5,0.871280,1.601172,75.423162
4,212,2,52.54,61071,39.724923,-89.094662,14381,42.227150,-90.645944,23.875300,4,0,4,1.051622,1.199672,79.158009


In [132]:
curr_data[["feature1", "feature2"]] = curr_data[["feature1", "feature2"]].astype("category")

In [133]:
curr_data.head()

,feature1,feature2,feature3,feature4,feature5,feature6,feature7,feature8,feature9,feature10,feature11,feature12,feature13,feature14,feature15,feature16
0,-1,12,28.71,37341,34.764269,-82.233754,12085,34.310798,-85.949851,23.232385,23,2,6,0.925699,1.657646,64.008021
1,7,11,4.56,48314,42.234590,-83.858655,129683,41.835677,-82.599450,21.750394,15,0,6,1.151222,1.674276,86.896538
2,197,4,43.01,62221,32.594352,-89.977493,95666,38.438088,-89.192691,15.009756,11,1,5,1.019024,1.355796,81.926642
3,288,9,8.73,31907,28.381704,-84.570469,173903,33.399397,-85.824090,23.088269,17,3,5,0.871280,1.601172,75.423162
4,212,2,52.54,61071,39.724923,-89.094662,14381,42.227150,-90.645944,23.875300,4,0,4,1.051622,1.199672,79.158009


In [134]:
curr_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 16 columns):
 #   Column     Non-Null Count  Dtype   
---  ------     --------------  -----   
 0   feature1   1000 non-null   category
 1   feature2   1000 non-null   category
 2   feature3   1000 non-null   float64 
 3   feature4   1000 non-null   int64   
 4   feature5   1000 non-null   float64 
 5   feature6   1000 non-null   float64 
 6   feature7   1000 non-null   int64   
 7   feature8   1000 non-null   float64 
 8   feature9   1000 non-null   float64 
 9   feature10  1000 non-null   float64 
 10  feature11  1000 non-null   int64   
 11  feature12  1000 non-null   int64   
 12  feature13  1000 non-null   int64   
 13  feature14  1000 non-null   float64 
 14  feature15  1000 non-null   float64 
 15  feature16  1000 non-null   float64 
dtypes: category(2), float64(9), int64(5)
memory usage: 118.3 KB


In [135]:
ref_data[["feature1", "feature2"]] = ref_data[["feature1", "feature2"]].astype("category")

In [137]:
ref_data_sample = ref_data.sample(n=1000, replace=False)

In [140]:
ref_data_sample

,feature1,feature2,feature3,feature4,feature5,feature6,feature7,feature8,feature9,feature10,feature11,feature12,feature13,feature14,feature15,feature16
64319,329,2,1.58,30345,35.715964,-87.388788,900273,33.849364,-83.642997,33.415036,7,2,1,8.017923,0.589755,69.570013
3201,448,12,87.64,78613,29.223939,-97.829266,65099,30.339897,-97.441101,43.501826,19,1,1,6.240134,0.473047,65.160890
4247,219,4,50.95,42286,37.639877,-86.266464,1291,37.415954,-86.422565,43.075840,3,0,1,7.941340,0.833768,66.714172
98454,143,9,3.62,65075,38.931179,-91.169182,784,39.018735,-92.303238,83.586533,1,4,1,6.610723,0.270250,60.255419
70930,352,6,2.65,76401,32.928420,-95.280092,26549,32.703632,-98.666997,32.049976,16,1,2,8.681461,0.509912,59.765998
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79150,42,8,102.76,19512,43.077151,-76.700799,16954,40.463962,-76.620898,51.392533,1,5,1,6.694004,0.525845,66.696518
68591,199,11,84.13,99218,44.679500,-118.562711,342856,48.161686,-116.685365,85.663044,11,4,1,6.044511,0.397564,70.120335
107038,394,3,40.40,38583,36.462832,-84.006856,23825,36.606398,-85.152076,39.011175,9,3,1,8.899809,0.618328,55.821286
44134,430,12,35.50,10466,39.804753,-71.808172,1382480,40.573597,-73.106375,28.817122,23,5,1,10.154460,0.624343,65.359463


In [139]:
report = Report(metrics=[
    DataDriftPreset(), 
])

report.run(reference_data=ref_data_sample, current_data=curr_data, column_mapping=column_mapping)
report

/home/toducthanh155/.local/share/virtualenvs/mlops-mara-sample-public-DwtKEUlZ/lib/python3.9/site-packages/scipy/stats/_stats_py.py:7407: RuntimeWarning:

divide by zero encountered in divide

/home/toducthanh155/.local/share/virtualenvs/mlops-mara-sample-public-DwtKEUlZ/lib/python3.9/site-packages/scipy/stats/_stats_py.py:7407: RuntimeWarning:

divide by zero encountered in divide



In [128]:
curr_data.head()

,feature1,feature2,feature3,feature4,feature5,feature6,feature7,feature8,feature9,feature10,feature11,feature12,feature13,feature14,feature15,feature16
0,-1,12,28.71,37341,34.764269,-82.233754,12085,34.310798,-85.949851,23.232385,23,2,6,0.925699,1.657646,64.008021
1,7,11,4.56,48314,42.234590,-83.858655,129683,41.835677,-82.599450,21.750394,15,0,6,1.151222,1.674276,86.896538
2,197,4,43.01,62221,32.594352,-89.977493,95666,38.438088,-89.192691,15.009756,11,1,5,1.019024,1.355796,81.926642
3,288,9,8.73,31907,28.381704,-84.570469,173903,33.399397,-85.824090,23.088269,17,3,5,0.871280,1.601172,75.423162
4,212,2,52.54,61071,39.724923,-89.094662,14381,42.227150,-90.645944,23.875300,4,0,4,1.051622,1.199672,79.158009


In [129]:
ref_data.head()

,feature1,feature2,feature3,feature4,feature5,feature6,feature7,feature8,feature9,feature10,feature11,feature12,feature13,feature14,feature15,feature16
0,337,0,138.71,44121,39.354345,-83.372015,769341,41.359975,-80.587306,78.529290,10,1,1,6.406360,0.284834,67.097437
1,365,12,6.51,66442,35.920414,-98.480588,14283,39.439880,-96.938809,45.234461,18,2,2,6.800447,0.779245,66.892996
2,184,6,3.34,42071,36.155589,-88.412050,29655,37.220467,-88.393033,43.063620,22,2,2,8.342428,0.603699,66.288586
3,390,4,180.17,30067,31.949179,-82.265798,302309,33.430411,-84.920539,26.093487,0,3,1,8.697487,0.472254,63.834991
4,365,10,8.92,66442,40.881803,-99.983715,14283,38.770383,-95.877356,44.831443,16,5,2,6.544843,0.429824,62.746348


In [130]:
curr_data.describe()

,feature1,feature2,feature3,feature4,feature5,feature6,feature7,feature8,feature9,feature10,feature11,feature12,feature13,feature14,feature15,feature16
count,1000.000000,1000.000000,1000.000000,1000.00000,1000.000000,1000.000000,1.000000e+03,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000
mean,181.629000,6.166000,63.932400,52735.58400,37.778901,-92.558009,3.259330e+05,37.816208,-92.669034,21.548949,14.612000,3.867000,6.123000,1.009690,1.471436,74.649172
std,159.833044,4.119395,161.900263,30183.65933,5.671083,17.004213,5.792758e+05,5.418617,16.916280,2.401407,6.163395,1.846814,1.419819,0.193584,0.273858,5.701141
min,-1.000000,0.000000,1.010000,1038.00000,20.327835,-157.581205,2.280000e+02,20.862999,-158.679801,13.931481,0.000000,0.000000,4.000000,0.455006,0.148857,50.264156
25%,28.000000,2.000000,6.545000,28403.00000,34.091153,-105.814168,2.300250e+04,33.938758,-105.324278,19.958469,12.000000,3.000000,5.000000,0.868054,1.343754,71.417314
50%,143.000000,6.000000,24.845000,49036.00000,38.478920,-87.191904,6.614000e+04,39.110726,-87.036115,21.829280,15.000000,4.000000,6.000000,1.003355,1.496621,74.975982
75%,333.000000,10.000000,61.640000,80515.00000,41.444732,-79.356814,3.211248e+05,41.335993,-79.765101,23.417276,20.000000,5.000000,7.000000,1.142895,1.640852,78.512143
max,478.000000,13.000000,2203.690000,99701.00000,64.712742,-67.137884,2.906700e+06,65.611064,-69.438122,25.903961,23.000000,6.000000,9.000000,1.632517,2.066790,89.656650


In [131]:
ref_data.describe()

,feature1,feature2,feature3,feature4,feature5,feature6,feature7,feature8,feature9,feature10,feature11,feature12,feature13,feature14,feature15,feature16
count,107360.000000,107360.000000,107360.000000,107360.000000,107360.000000,107360.000000,1.073600e+05,107360.000000,107360.000000,107360.000000,107360.000000,107360.000000,107360.000000,107360.000000,107360.000000,107360.000000
mean,240.049478,6.398566,98.882623,50738.075354,37.240961,-91.853114,2.940327e+05,37.239563,-91.860593,48.306559,12.706501,3.480086,1.502459,7.496707,0.531900,66.232903
std,139.267539,3.920962,198.289371,29606.912130,5.718932,16.355590,5.529317e+05,5.383233,16.241312,16.677811,7.059975,1.991065,0.536569,1.997109,0.234868,5.265216
min,0.000000,0.000000,1.000000,1106.000000,11.873034,-172.649290,1.940000e+02,18.798261,-169.019669,23.447657,0.000000,0.000000,1.000000,-1.003550,-0.335346,44.152711
25%,121.000000,3.000000,9.940000,28152.000000,33.378770,-98.953357,1.671900e+04,33.609103,-98.089413,34.976274,7.000000,2.000000,1.000000,6.151884,0.375587,62.759422
50%,236.000000,6.000000,48.690000,46222.000000,37.492329,-87.146479,6.200900e+04,37.560009,-86.949154,45.014083,13.000000,4.000000,1.000000,7.499778,0.517131,66.313658
75%,363.000000,10.000000,90.990000,78045.000000,41.138940,-79.588290,2.475300e+05,40.978207,-80.001092,58.726041,19.000000,5.000000,2.000000,8.844669,0.667085,69.793228
max,480.000000,13.000000,9875.470000,99791.000000,76.845878,-63.066068,2.906700e+06,71.485302,-69.133860,96.996128,23.000000,6.000000,3.000000,16.812962,1.703339,88.834367
